In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd

class VehicleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
        
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')  # 读取图像并转换为RGB格式
        if self.transform:
            image = self.transform(image)  # 如果提供了transform就进行预处理
        label = self.labels[idx]  # 获取对应的标签
        return image, label  # 返回图像张量和标签

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, val_loader, device, verbose=False):
    model.eval()
    all_preds = []
    all_labels = []
    batch_accuracies = []
    batch_f1_scores = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            batch_preds = preds.cpu().numpy()
            batch_labels = labels.cpu().numpy()
            
            # 计算每批次的accuracy和macro-F1
            if len(np.unique(batch_labels)) > 1:  # 确保批次中至少有两个不同的类别
                batch_accuracy = accuracy_score(batch_labels, batch_preds)
                batch_f1 = f1_score(batch_labels, batch_preds, average='macro', zero_division=1)
                batch_accuracies.append(batch_accuracy)
                batch_f1_scores.append(batch_f1)
            
            all_preds.extend(batch_preds)
            all_labels.extend(batch_labels)
    
    # 计算整体metrics
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    
    # 计算每批次的方差（如果有足够的批次）
    accuracy_variance = np.var(batch_accuracies) if len(batch_accuracies) > 1 else 0.0
    f1_variance = np.var(batch_f1_scores) if len(batch_f1_scores) > 1 else 0.0
    
    # 在 evaluate 函数中的打印信息修改
    if verbose and len(batch_accuracies) > 0:
        print(f"\n批次级别统计:")
        print(f"批次数量: {len(batch_accuracies)}")
        print(f"批次准确率: {[f'{acc:.4f}' for acc in batch_accuracies]}")
        print(f"批次宏F1分数: {[f'{f1:.4f}' for f1 in batch_f1_scores]}")
        print(f"批次准确率均值: {np.mean(batch_accuracies):.4f}, 方差: {accuracy_variance:.4f}")
        print(f"批次宏F1均值: {np.mean(batch_f1_scores):.4f}, 方差: {f1_variance:.4f}")

    return accuracy, macro_f1, accuracy_variance, f1_variance

In [2]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # 加载数据
    data_dir = "Data/Data"  # 修正数据路径
    image_paths = []
    labels = []
    
    # 遍历文件夹收集图片路径和标签
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # 从文件名中提取类别标签（1_xxx.png -> 0, 2_xxx.png -> 1, 3_xxx.png -> 2）
            class_id = int(filename.split('_')[0]) - 1  # 将1,2,3映射到0,1,2
            image_paths.append(os.path.join(data_dir, filename))
            labels.append(class_id)
    
    # 转换为numpy数组
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    print(f"总样本数: {len(labels)}")
    print("类别分布:")
    for i in range(3):
        print(f"类别 {i+1}: {sum(labels == i)} 个样本")
    
    # 5折交叉验证
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    all_batch_accuracies = []  # 存储所有fold的batch准确率
    all_batch_f1_scores = []   # 存储所有fold的batch F1分数
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
        print(f"\nFold {fold + 1}")
        
        # 准备数据集
        train_dataset = VehicleDataset(
            [image_paths[i] for i in train_idx],
            [labels[i] for i in train_idx],
            transform=transform
        )
        val_dataset = VehicleDataset(
            [image_paths[i] for i in val_idx],
            [labels[i] for i in val_idx],
            transform=transform
        )
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # 模型初始化
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 3)  # 3个类别
        model = model.to(device)
        
        # 损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_score = 0
        best_epoch = 0
        patience = 3
        no_improve = 0
        
        # 训练循环 (修正缩进)
        for epoch in range(15):
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            accuracy, macro_f1, accuracy_variance, f1_variance = evaluate(model, val_loader, device, verbose=False)
            
            current_score = 0.7 * accuracy + 0.3 * macro_f1
            
            print(f"轮次 {epoch+1}, 损失: {train_loss:.4f}, 准确率: {accuracy:.4f}, 宏F1: {macro_f1:.4f}, 得分: {current_score:.4f}")
            
            if current_score > best_score:
                best_score = current_score
                best_epoch = epoch
                no_improve = 0
                torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"在轮次 {epoch+1} 触发早停")
                    break
        
        # 重新加载当前fold的最佳模型再评估
        model.load_state_dict(torch.load(f'best_model_fold{fold}.pth'))
        
        # 记录最终结果（使用verbose=True来显示batch级别的结果）
        final_accuracy, final_macro_f1, accuracy_variance, f1_variance = evaluate(model, val_loader, device, verbose=True)
        accuracies.append(final_accuracy)
        f1_scores.append(final_macro_f1)
        
        print(f"\n第 {fold + 1} 折最终结果:")
        print(f"Accuracy: {final_accuracy:.4f}")
        print(f"Macro-F1: {final_macro_f1:.4f}")
        print(f"Accuracy Variance: {accuracy_variance:.4f}")
        print(f"Macro-F1 Variance: {f1_variance:.4f}")
        fold_score = 0.7 * final_accuracy + 0.3 * final_macro_f1
        print(f"本折得分: {fold_score:.4f}")

    # 计算5折交叉验证的平均结果
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)  # 使用标准差代替方差
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)  # 使用标准差代替方差
    
    print("\n最终五折交叉验证结果与指标:")
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Macro-F1: {mean_f1:.4f} ± {std_f1:.4f}")
    
    # 计算最终平均得分
    final_score = 0.7 * mean_accuracy + 0.3 * mean_f1
    print(f"\n最终平均得分: {final_score:.4f}")

if __name__ == "__main__":
    main()

使用设备: cuda
总样本数: 1572
类别分布:
类别 1: 720 个样本
类别 2: 608 个样本
类别 3: 244 个样本

Fold 1


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


轮次 1, 损失: 0.2284, 准确率: 0.7429, 宏F1: 0.7057, 得分: 0.7317
轮次 2, 损失: 0.1918, 准确率: 0.9460, 宏F1: 0.9403, 得分: 0.9443
轮次 3, 损失: 0.0742, 准确率: 0.8571, 宏F1: 0.8636, 得分: 0.8591
轮次 4, 损失: 0.1315, 准确率: 0.9270, 宏F1: 0.9263, 得分: 0.9268
轮次 5, 损失: 0.0856, 准确率: 0.9143, 宏F1: 0.9063, 得分: 0.9119
在轮次 5 触发早停

批次级别统计:
批次数量: 2
批次准确率: ['0.9688', '1.0000']
批次宏F1分数: ['0.9687', '1.0000']
批次准确率均值: 0.9844, 方差: 0.0002
批次宏F1均值: 0.9844, 方差: 0.0002

第 1 折最终结果:
Accuracy: 0.9429
Macro-F1: 0.9426
Accuracy Variance: 0.0002
Macro-F1 Variance: 0.0002
本折得分: 0.9428

Fold 2


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


轮次 1, 损失: 0.2856, 准确率: 0.8190, 宏F1: 0.7149, 得分: 0.7878
轮次 2, 损失: 0.1331, 准确率: 0.8127, 宏F1: 0.7871, 得分: 0.8050
轮次 3, 损失: 0.0912, 准确率: 0.9683, 宏F1: 0.9616, 得分: 0.9663
轮次 4, 损失: 0.0768, 准确率: 0.9175, 宏F1: 0.8825, 得分: 0.9070
轮次 5, 损失: 0.0669, 准确率: 0.8762, 宏F1: 0.8542, 得分: 0.8696
轮次 6, 损失: 0.0418, 准确率: 0.9302, 宏F1: 0.9275, 得分: 0.9294
在轮次 6 触发早停

批次级别统计:
批次数量: 2
批次准确率: ['0.9688', '0.9688']
批次宏F1分数: ['0.6559', '0.6589']
批次准确率均值: 0.9688, 方差: 0.0000
批次宏F1均值: 0.6574, 方差: 0.0000

第 2 折最终结果:
Accuracy: 0.9683
Macro-F1: 0.9597
Accuracy Variance: 0.0000
Macro-F1 Variance: 0.0000
本折得分: 0.9657

Fold 3


...

轮次 1, 损失: 0.2854, 准确率: 0.8949, 宏F1: 0.8897, 得分: 0.8933
轮次 2, 损失: 0.1539, 准确率: 0.9459, 宏F1: 0.9407, 得分: 0.9443
轮次 3, 损失: 0.0762, 准确率: 0.9172, 宏F1: 0.8713, 得分: 0.9034
轮次 4, 损失: 0.1313, 准确率: 0.9745, 宏F1: 0.9775, 得分: 0.9754
轮次 5, 损失: 0.0832, 准确率: 0.9745, 宏F1: 0.9756, 得分: 0.9748
轮次 6, 损失: 0.0753, 准确率: 0.9777, 宏F1: 0.9706, 得分: 0.9756
轮次 7, 损失: 0.0341, 准确率: 0.9618, 宏F1: 0.9536, 得分: 0.9593
轮次 8, 损失: 0.0286, 准确率: 0.9713, 宏F1: 0.9640, 得分: 0.9691
轮次 9, 损失: 0.0948, 准确率: 0.8503, 宏F1: 0.8299, 得分: 0.8442
在轮次 9 触发早停

批次级别统计:
批次数量: 2
批次准确率: ['0.9375', '0.9688']
批次宏F1分数: ['0.6444', '0.6589']
批次准确率均值: 0.9531, 方差: 0.0002
批次宏F1均值: 0.6517, 方差: 0.0001

第 3 折最终结果:
Accuracy: 0.9777
Macro-F1: 0.9703
Accuracy Variance: 0.0002
Macro-F1 Variance: 0.0001
本折得分: 0.9755

Fold 4


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


轮次 1, 损失: 0.2684, 准确率: 0.9172, 宏F1: 0.9045, 得分: 0.9134
轮次 2, 损失: 0.1811, 准确率: 0.9140, 宏F1: 0.8963, 得分: 0.9087
轮次 3, 损失: 0.1065, 准确率: 0.8854, 宏F1: 0.8775, 得分: 0.8830
轮次 4, 损失: 0.0996, 准确率: 0.9363, 宏F1: 0.9288, 得分: 0.9340
轮次 5, 损失: 0.0606, 准确率: 0.9427, 宏F1: 0.9394, 得分: 0.9417
轮次 6, 损失: 0.0395, 准确率: 0.9204, 宏F1: 0.8903, 得分: 0.9113
轮次 7, 损失: 0.0359, 准确率: 0.9777, 宏F1: 0.9712, 得分: 0.9758
轮次 8, 损失: 0.0655, 准确率: 0.9459, 宏F1: 0.9441, 得分: 0.9453
轮次 9, 损失: 0.1117, 准确率: 0.9427, 宏F1: 0.9322, 得分: 0.9395
轮次 10, 损失: 0.0726, 准确率: 0.9427, 宏F1: 0.9357, 得分: 0.9406
在轮次 10 触发早停

批次级别统计:
批次数量: 2
批次准确率: ['0.9688', '0.9062']
批次宏F1分数: ['0.9687', '0.6259']
批次准确率均值: 0.9375, 方差: 0.0010
批次宏F1均值: 0.7973, 方差: 0.0294

第 4 折最终结果:
Accuracy: 0.9650
Macro-F1: 0.9531
Accuracy Variance: 0.0010
Macro-F1 Variance: 0.0294
本折得分: 0.9614

Fold 5


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


轮次 1, 损失: 0.2510, 准确率: 0.8057, 宏F1: 0.7752, 得分: 0.7966
轮次 2, 损失: 0.1745, 准确率: 0.9459, 宏F1: 0.9395, 得分: 0.9439
轮次 3, 损失: 0.0751, 准确率: 0.9713, 宏F1: 0.9619, 得分: 0.9685
轮次 4, 损失: 0.1190, 准确率: 0.7420, 宏F1: 0.7433, 得分: 0.7424
轮次 5, 损失: 0.1040, 准确率: 0.9650, 宏F1: 0.9550, 得分: 0.9620
轮次 6, 损失: 0.0736, 准确率: 0.8185, 宏F1: 0.7373, 得分: 0.7941
在轮次 6 触发早停

批次级别统计:
批次数量: 2
批次准确率: ['1.0000', '0.9688']
批次宏F1分数: ['1.0000', '0.6593']
批次准确率均值: 0.9844, 方差: 0.0002
批次宏F1均值: 0.8296, 方差: 0.0290

第 5 折最终结果:
Accuracy: 0.9809
Macro-F1: 0.9718
Accuracy Variance: 0.0002
Macro-F1 Variance: 0.0290
本折得分: 0.9782

最终五折交叉验证结果与指标:
Accuracy: 0.9669 ± 0.0134
Macro-F1: 0.9595 ± 0.0109

最终平均得分: 0.9647
